In [ ]:
# -*- coding: utf-8 -*-
"""cefar10_neural_network.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1v0IrInTE6cr3aodq9TA2uR2KP3hCra4o
"""

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def to_categorical(x, n_col=None):
    if not n_col:
        n_col = np.amax(x) + 1
    one_hot = np.zeros((x.shape[0], n_col))
    one_hot[np.arange(x.shape[0]), x] = 1
    return one_hot

def accuracy(y_true, y_pred):
  return np.sum(y_true == y_pred, axis=0) / len(y_true)

def batch_loader(X, y = None, batch_size=64):
  n_samples = X.shape[0]
  for i in np.arange(0, n_samples, batch_size):
    begin, end = i, min(i + batch_size, n_samples)
    if y is not None:
      yield X[begin:end], y[begin: end]
    else:
      yield X[begin:end]

def showGraph(cost):
    plt.grid()
    plt.plot([100 * val for val in range(len(cost))], cost)
    plt.title('Accuracy')
    plt.xlabel('Training Iterations')
    plt.ylabel('Accuracy')
    plt.show()

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = y_train.reshape(y_train.shape[0],), y_test.reshape(y_test.shape[0],)
y_train, y_test = to_categorical(y_train.astype("int"), 10), to_categorical(y_test.astype("int"))
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape(-1, 32*32*3), X_test.reshape(-1, 32*32*3)
n_input_dim = 32*32*3 
n_out = 10 

class CrossEntropy():
    def __init__(self): pass

    def loss(self, y, p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return -y * np.log(p) - (1 - y) * np.log(1 - p)

    def gradient(self, y, p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return -(y / p) + (1 - y) / (1 - p)

class LeakyReLU():
    def __init__(self, alpha=0.2):
        self.alpha = alpha

    def __call__(self, x):
        return self.activation(x)

    def activation(self, x):
        return np.where(x >= 0, x, self.alpha * x)

    def gradient(self, x):
        return np.where(x >= 0, 1, self.alpha)

class ReLU:
  def __init__(self): pass

  def __call__(self, x):
        return self.activation(x)

  def activation(self, x):
        return np.maximum(0.0, x)

  def gradient(self, x):
        return np.where(x >= 0, 1, 0)

class Softmax():
    def __init__(self): pass

    def __call__(self, x):
        return self.activation(x)

    def activation(self, x):
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return e_x / np.sum(e_x, axis=-1, keepdims=True)

    def gradient(self, x):
        p = self.activation(x)
        return p * (1 - p)

class Activation():

    def __init__(self, activation, name="activation"):
        self.activation = activation
        self.gradient = activation.gradient
        self.input = None
        self.output = None
        self.name = name

    def forward(self, x):
        self.input = x
        self.output = self.activation(x)
        return self.output

    def backward(self, output_error, lr=0.01):
        return self.gradient(self.input)*output_error

    def __call__(self, x):
        return self.forward(x)

class Linear():
    def __init__(self, n_in, n_out, name="linear"):
        limit = 1 / np.sqrt(n_in)
        self.W = np.random.uniform(-limit, limit, (n_in, n_out))
        self.b = np.zeros((1, n_out))  # Biases
        self.vw, self.sw = np.zeros(self.W.shape), np.zeros(self.W.shape)
        self.vb, self.sb = np.zeros(self.b.shape), np.zeros(self.b.shape)
        self.input = None
        self.output = None
        self.name = name

    def forward(self, x):
        l, c = x.shape
        d = np.random.rand(l, c) < 0.9
        x = np.multiply(x, d)
        self.input = x/0.9
        self.output = np.dot(self.input, self.W) + self.b  # Wx + b
        return self.output

    def backward(self, output_error, lr=0.01):
        input_error = np.dot(output_error, self.W.T)
        delta = np.dot(self.input.T, output_error)
        self.sw = 0.999 * self.sw + 0.001*np.power(delta, 2)
        self.sb = 0.999 * self.sb + 0.001*np.power(np.mean(output_error), 2)
        """self.vw = 0.9 * self.vw + 0.1*delta
        self.vb = 0.9 * self.vb + 0.1*np.mean(output_error)
        sb_cor, vb_cor = self.sb/0.001, self.vb/0.1
        sw_cor, vw_cor = self.sw/0.001, self.vw/0.1
        self.W -= (lr/np.sqrt(sw_cor+1e-8))*vw_cor
        self.b -= (lr/np.sqrt(sb_cor+1e-8))*vb_cor"""
        self.W -= (lr/np.sqrt(self.sw+1e-8))*delta
        self.b = (lr/np.sqrt(self.sb+1e-8))*np.mean(output_error)
        return input_error
    def __call__(self, x):
        return self.forward(x)
    def set_layer(self, weights, biases):
        self.W = weights
        self.b = biases

class Network():
    def __init__(self, input_dim, output_dim, lr=0.01):
        # input_dim = 3072, output_dim = 10 for cifar10
        self.layers = [
            Linear(input_dim, 1024, name="input"),
            Activation(LeakyReLU(), name="arelu1"),
            Linear(1024, 512, name="input"),
            Activation(LeakyReLU(), name="arelu2"),
            Linear(512, 256, name="input"),
            Activation(LeakyReLU(), name="arelu3"),
            Linear(256, 128, name="input"),
            Activation( LeakyReLU(), name="arelu4"),
            Linear(128, output_dim, name="output"),
            Activation(Softmax(), name="softmax")
        ]
        self.lr = lr

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def backward(self, loss_grad):
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad, self.lr)

    def __call__(self, x):
        return self.forward(x)

def train(n_input_dim, n_out,EPOCHS=5,  lr=1e-2):
    criterion = CrossEntropy()
    model = Network(n_input_dim, n_out, lr=1e-2)
    cost = []
    for epoch in range(EPOCHS):
        loss = []
        acc = []
        counter = 0
        for x_batch, y_batch in batch_loader(X_train, y_train):
            counter += 1
            out = model(x_batch)
            loss.append(np.mean(criterion.loss(y_batch, out)))
            acc.append(accuracy(np.argmax(y_batch, axis=1), np.argmax(out, axis=1)))
            if counter % 100 == 0:
                tmp = np.array(acc)
                cost.append(100*np.mean(tmp[counter-100: counter]))
            error = criterion.gradient(y_batch, out)
            model.backward(error)
        print(f"Epoch {epoch + 1}, Loss: {np.mean(loss)*100:.2f} %, Accuracy: {np.mean(acc)*100:.2f} %")
    return model, cost

def test(model, X, y):
    print("-------- Test --------")
    for i, val in enumerate(X):
        out = model(val)
        y_target = np.argmax(y[i])
        y_predict = np.argmax(out)
        #print(f"y_predict: {y_predict} ---- y: {y_target}")
    acc = accuracy(np.argmax(y, axis=1), np.argmax(model(X), axis=1))*100
    print("----------------------")
    print(f"Test Accuracy: {acc:.2f} %")

model, cost = train(n_input_dim, n_out, 100)

showGraph(cost)

test(model, X_train[:5000], y_train[:5000])